# Extract features from cleaned EEG Data

In [4]:
import mne
import os
from tqdm import tqdm_notebook
import numpy as np
from scipy.signal import welch

In [5]:
import warnings
import logging
import pandas as pd

warnings.filterwarnings("ignore")

logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)
logging.getLogger("mne").setLevel(logging.WARNING)

In [6]:
pd.read_excel("ratings/Result movies.xlsx")

,Participant,a.mp4,b.mp4,c.mp4,d.mp4,e.mp4,f.mp4,g.mp4,h.mp4,Unnamed: 9,порядок предъявления,Unnamed: 11,Unnamed: 12
0,1,10,10.000000,2.000000,5.000000,9.000000,9.000000,5.000000,9.000000,NaN,14853762,NaN,NaN
1,2,10,8.000000,7.000000,9.000000,7.000000,6.000000,6.000000,9.000000,NaN,26753841,NaN,NaN
2,3,6,3.000000,5.000000,6.000000,5.000000,5.000000,5.000000,2.000000,NaN,83261475,NaN,NaN
3,4,8,6.000000,5.000000,9.000000,5.000000,7.000000,4.000000,6.000000,NaN,57164283,NaN,NaN
4,5,3,2.000000,4.000000,1.000000,7.000000,3.000000,7.000000,2.000000,NaN,72481536,NaN,NaN
5,6,7,7.000000,2.000000,8.000000,5.000000,3.000000,2.000000,5.000000,NaN,14853762,NaN,NaN
6,7,9,4.000000,4.000000,3.000000,9.000000,8.000000,8.000000,5.000000,NaN,26753841,NaN,NaN
7,8,5,7.000000,5.000000,9.000000,3.000000,2.000000,8.000000,7.000000,NaN,83261475,NaN,NaN
8,9,8,8.000000,2.000000,3.000000,7.000000,5.000000,5.000000,9.000000,NaN,57164283,NaN,NaN
9,10,3,8.000000,7.000000,3.000000,1.000000,3.000000,6.000000,2.000000,NaN,72481536,NaN,NaN


In [7]:
ratings = pd.read_excel("ratings/Result movies.xlsx")
ratings = ratings[0:21][
    ["a.mp4", "b.mp4", "c.mp4", "d.mp4", "e.mp4", "f.mp4", "g.mp4", "h.mp4"]
]
ratings

,a.mp4,b.mp4,c.mp4,d.mp4,e.mp4,f.mp4,g.mp4,h.mp4
0,10,10.0,2.0,5.0,9.0,9.0,5.0,9.0
1,10,8.0,7.0,9.0,7.0,6.0,6.0,9.0
2,6,3.0,5.0,6.0,5.0,5.0,5.0,2.0
3,8,6.0,5.0,9.0,5.0,7.0,4.0,6.0
4,3,2.0,4.0,1.0,7.0,3.0,7.0,2.0
5,7,7.0,2.0,8.0,5.0,3.0,2.0,5.0
6,9,4.0,4.0,3.0,9.0,8.0,8.0,5.0
7,5,7.0,5.0,9.0,3.0,2.0,8.0,7.0
8,8,8.0,2.0,3.0,7.0,5.0,5.0,9.0
9,3,8.0,7.0,3.0,1.0,3.0,6.0,2.0


In [5]:
watch = mne.io.read_raw("021.vhdr")

# Extract Complexity Features (neurokit2)

In [ ]:
import neurokit2 as nk

In [12]:
n_subjects = 21
n_films = 8
n_channels = 18
channel_names = [
    "Fz",
    "F3",
    "F7",
    "C3",
    "T7",
    "Pz",
    "P3",
    "P7",
    "O1",
    "Oz",
    "O2",
    "P4",
    "P8",
    "Cz",
    "C4",
    "T8",
    "F4",
    "F8",
]
for Subject in tqdm_notebook(range(1, n_subjects + 1)):
    if Subject == 5 or Subject == 21:
        continue
    for film in ["a", "b", "c", "d", "e", "f", "g", "h"]:
        if not os.path.isfile("cropped_new/{}/watch_{}.fif".format(Subject, film)):
            continue

        raw = mne.io.read_raw("cropped_new/{}/watch_{}.fif".format(Subject, film))

        if "VEOG" in raw.ch_names:
            raw.drop_channels(["VEOG"])
        raw.drop_channels(["Pulse", "Zygom", "Corr", "EDA", "Mark"])

        for channel in range(n_channels):
            df, info = nk.complexity(raw[channel_names[channel]][0][0], which=["fast"])
            df["Subj"] = Subject
            df["film"] = ord(film) - 96
            df["ch"] = channel + 1
            df["labels"] = ratings.loc[Subject - 1][film + ".mp4"]
            df.to_excel(
                "features_neurokit/S_{}_F_{}_{}_.xlsx".format(
                    Subject, film, channel_names[channel]
                ),
                index=False,
            )

  0%|          | 0/1 [00:00<?, ?it/s]

# Extract Spectral Features (Yasa)

In [ ]:
import yasa

In [13]:
n_subjects = 21
n_films = 8
n_channels = 18
channel_names = [
    "Fz",
    "F3",
    "F7",
    "C3",
    "T7",
    "Pz",
    "P3",
    "P7",
    "O1",
    "Oz",
    "O2",
    "P4",
    "P8",
    "Cz",
    "C4",
    "T8",
    "F4",
    "F8",
]
for Subject in tqdm_notebook(range(1, n_subjects)):
    frames = []
    frames_rel = []
    if Subject == 5:
        continue
    for film in ["a", "b", "c", "d", "e", "f", "g", "h"]:
        relax = mne.io.read_raw("cropped/{}/relax_{}.fif".format(Subject, film))
        watch = mne.io.read_raw("cropped/{}/watch_{}.fif".format(Subject, film))
        sf = relax.info["sfreq"]
        window = int(sf)
        if "VEOG" in relax.ch_names:
            relax.drop_channels(["VEOG"])
            watch.drop_channels(["VEOG"])
        relax.drop_channels(["Pulse", "Zygom", "Corr", "EDA", "Mark"])
        watch.drop_channels(["Pulse", "Zygom", "Corr", "EDA", "Mark"])

        freqs_relax, psd_relax = welch(
            relax.get_data() * 1e6, sf, nperseg=window, average="median"
        )
        freqs_watch, psd_watch = welch(
            watch.get_data() * 1e6, sf, nperseg=window, average="median"
        )

        bp_relax = yasa.bandpower_from_psd(
            psd_relax,
            freqs_relax,
            relax.ch_names,
            bands=[(4, 8, "Theta"), (8, 12, "Alpha"), (12, 30, "Beta")],
            relative=False,
        ).drop(columns=["FreqRes", "Relative", "TotalAbsPow"])
        bp_watch = yasa.bandpower_from_psd(
            psd_watch,
            freqs_watch,
            watch.ch_names,
            bands=[(4, 8, "Theta"), (8, 12, "Alpha"), (12, 30, "Beta")],
            relative=False,
        ).drop(columns=["FreqRes", "Relative", "TotalAbsPow"])
        bp_ratio_relax = yasa.bandpower(
            relax,
            relax.info["sfreq"],
            bands=[
                (4, 8, "Theta_ratio"),
                (8, 12, "Alpha_ratio"),
                (12, 30, "Beta_ratio"),
            ],
            relative=True,
        ).drop(columns=["FreqRes", "Relative"])
        bp_ratio_watch = yasa.bandpower(
            watch,
            watch.info["sfreq"],
            bands=[
                (4, 8, "Theta_ratio"),
                (8, 12, "Alpha_ratio"),
                (12, 30, "Beta_ratio"),
            ],
            relative=True,
        ).drop(columns=["FreqRes", "Relative"])
        for channel in range(n_channels):
            df_relax = pd.DataFrame(bp_relax.loc[channel]).transpose()
            df_watch = pd.DataFrame(bp_watch.loc[channel]).transpose()
            dr = pd.DataFrame(bp_ratio_relax.loc[relax.ch_names[channel]]).transpose()
            dr.rename(index={relax.ch_names[channel]: channel}, inplace=True)
            df_relax = df_relax.join(dr)
            dw = pd.DataFrame(bp_ratio_watch.loc[watch.ch_names[channel]]).transpose()
            dw.rename(index={watch.ch_names[channel]: channel}, inplace=True)
            df_watch = df_watch.join(dw)
            df_relax.rename(index={channel: "F" + str(film) + "_relax"}, inplace=True)
            df_watch.rename(index={channel: "F" + str(film) + "_watch"}, inplace=True)
            df_rel = {
                "Theta_watch-relax": float(df_watch["Theta"])
                - float(df_relax["Theta"]),
                "Alpha_watch-relax": float(df_watch["Alpha"])
                - float(df_relax["Alpha"]),
                "Beta_watch-relax": float(df_watch["Beta"]) - float(df_relax["Beta"]),
            }
            df_rel = pd.DataFrame(
                df_rel, index=["F" + str(film) + "_" + watch.ch_names[channel]]
            )
            df_div = {
                "Beta/Alpha": float(df_rel["Beta_watch-relax"])
                / float(df_rel["Alpha_watch-relax"]),
                "Beta/(Alpha + Theta)": float(df_rel["Beta_watch-relax"])
                / (
                    float(
                        df_rel["Alpha_watch-relax"] + float(df_rel["Theta_watch-relax"])
                    )
                ),
                "Beta_watch/Alpha_watch": float(df_watch["Beta"])
                / float(df_watch["Alpha"]),
                "Beta_watch/(Alpha_watch + Theta_watch)": float(df_watch["Beta"])
                / (float(df_watch["Alpha"] + float(df_watch["Theta"]))),
                "Beta_watch/Alpha_watch - Beta_relax/Alpha_relax": float(
                    df_watch["Beta"]
                )
                / float(df_watch["Alpha"])
                - float(df_relax["Beta"]) / float(df_relax["Alpha"]),
                "Beta_watch/(Alpha_watch + Theta_watch) - Beta_relax/(Alpha_relax + Theta_relax)": float(
                    df_watch["Beta"]
                )
                / (float(df_watch["Alpha"] + float(df_watch["Theta"])))
                - float(df_relax["Beta"])
                / (float(df_relax["Alpha"] + float(df_relax["Theta"]))),
            }
            df_div = pd.DataFrame(
                df_div, index=["F" + str(film) + "_" + watch.ch_names[channel]]
            )
            df_all = df_rel.join(df_div)
            frames_rel.append(df_all)
            frames.append(df_relax)
            frames.append(df_watch)
    result_rel = pd.concat(frames_rel)
    result_rel.to_excel("features_yasa/S" + str(Subject) + "_ratio.xlsx")
    result = pd.concat(frames)
    result.to_excel("features_yasa_/S" + str(Subject) + ".xlsx")

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
frames_rel = []
frames = []
bio = []
for Subject in tqdm_notebook(range(1, 21)):
    if Subject == 5:
        continue
    for film in ["a", "b", "c", "d", "e", "f", "g", "h"]:
        if not os.path.isfile("cropped_new/{}/watch_{}.fif".format(Subject, film)):
            continue
        relax = mne.io.read_raw("cropped_new/{}/relax_{}.fif".format(Subject, film))
        watch = mne.io.read_raw("cropped_new/{}/watch_{}.fif".format(Subject, film))
        sf = relax.info["sfreq"]
        win = int(sf)

        if "VEOG" in relax.ch_names:
            relax.drop_channels(["VEOG"])
            watch.drop_channels(["VEOG"])
        relax.drop_channels(["Pulse", "Zygom", "Corr", "EDA", "Mark"])
        watch.drop_channels(["Pulse", "Zygom", "Corr", "EDA", "Mark"])
        ch_names = relax.info["ch_names"]
        sf = relax.info["sfreq"]
        win = int(sf)
        freqs_watch, psd_watch = welch(
            watch.get_data() * 1e6, sf, nperseg=win, average="median"
        )
        freqs_relax, psd_relax = welch(
            relax.get_data() * 1e6, sf, nperseg=win, average="median"
        )
        bp_relax = yasa.bandpower_from_psd(
            psd_relax,
            freqs_relax,
            relax.ch_names,
            bands=[(4, 8, "Theta"), (8, 12, "Alpha"), (12, 30, "Beta")],
            relative=False,
        ).drop(columns=["FreqRes", "Relative", "TotalAbsPow"])
        bp_watch = yasa.bandpower_from_psd(
            psd_watch,
            freqs_watch,
            watch.ch_names,
            bands=[(4, 8, "Theta"), (8, 12, "Alpha"), (12, 30, "Beta")],
            relative=False,
        ).drop(columns=["FreqRes", "Relative", "TotalAbsPow"])

        df_all = {}
        df_F3_relax = pd.DataFrame(bp_relax.loc[1]).transpose()
        df_F3_watch = pd.DataFrame(bp_watch.loc[1]).transpose()
        df_F4_relax = pd.DataFrame(bp_relax.loc[16]).transpose()
        df_F4_watch = pd.DataFrame(bp_watch.loc[16]).transpose()
        B3_R = float(df_F3_relax["Beta"])
        B3_W = float(df_F3_watch["Beta"])
        B4_R = float(df_F4_relax["Beta"])
        B4_W = float(df_F4_watch["Beta"])
        A3_R = float(df_F3_relax["Alpha"])
        A3_W = float(df_F3_watch["Alpha"])
        A4_R = float(df_F4_relax["Alpha"])
        A4_W = float(df_F4_watch["Alpha"])
        df_rel = {
            "(B3+B4)/(A3+A4)": (B3_W + B4_W) / (A3_W + A4_W),
            "(A4/B4)-(A3/B3)": (A4_W / B4_W) - (A3_W / B3_W),
            "(B3+B4)W/(A3+A4)W-(B3+B4)R/(A3+A4)R": (B3_W + B4_W) / (A3_W + A4_W)
            - (B3_R + B4_R) / (A3_R + A4_R),
            "(A4/B4)W-(A3/B3)W-(A4+B4)R+(A3/B3)R": (A4_W / B4_W)
            - (A3_W / B3_W)
            - (A4_R / B4_R)
            + (A3_R / B3_R),
            "Subj": Subject,
            "film": ord(film) - 96,
        }
        frames_rel.append(df_rel)
bio = pd.DataFrame(frames_rel)

In [14]:
n_subjects = 21
n_channels = 18
ind = 0
ch_names = [
    "Fz",
    "F3",
    "F7",
    "C3",
    "T7",
    "Pz",
    "P3",
    "P7",
    "O1",
    "Oz",
    "O2",
    "P4",
    "P8",
    "Cz",
    "C4",
    "T8",
    "F4",
    "F8",
]
data_all = []
for Subject in tqdm_notebook(range(4, 5)):
    if Subject == 5:
        continue
    df_rel = pd.read_excel("features_yasa/S{}_ratio.xlsx".format(Subject))
    df = pd.read_excel("features_yasa/S{}.xlsx".format(Subject))
    first = 0
    second = 1
    for film in ["a", "b", "c", "d", "e", "f", "g", "h"]:
        for channel in range(n_channels):
            dp = df_rel.loc[
                (df_rel["Unnamed: 0"] == "F" + str(film) + "_" + ch_names[channel])
            ].rename(index={first: ind})
            d = df.loc[
                (df["Chan"] == ch_names[channel])
                & (df["Unnamed: 0"] == "F" + str(film) + "_watch")
            ].rename(index={second: ind})
            if dp.empty or d.empty:
                print("{}{}{}".format(Subject, film, ch_names[channel]))
                continue
            d["Theta_watch-relax"] = float(dp["Theta_watch-relax"])
            d["Alpha_watch-relax"] = float(dp["Alpha_watch-relax"])
            d["Beta_watch-relax"] = float(dp["Beta_watch-relax"])
            d["Beta/Alpha"] = float(dp["Beta/Alpha"])
            d["Beta/(Alpha + Theta)"] = float(dp["Beta/(Alpha + Theta)"])
            d["Beta_watch/Alpha_watch"] = float(dp["Beta_watch/Alpha_watch"])
            d["Beta_watch/(Alpha_watch + Theta_watch)"] = float(
                dp["Beta_watch/(Alpha_watch + Theta_watch)"]
            )
            d["Beta_watch/Alpha_watch - Beta_relax/Alpha_relax"] = float(
                dp["Beta_watch/Alpha_watch - Beta_relax/Alpha_relax"]
            )
            d[
                "Beta_watch/(Alpha_watch + Theta_watch) - Beta_relax/(Alpha_relax + Theta_relax)"
            ] = float(
                dp[
                    "Beta_watch/(Alpha_watch + Theta_relax) - Beta_relax/(Alpha_relax + Theta_relax)"
                ]
            )
            d = d.drop(["Unnamed: 0", "Chan"], axis=1)
            d["Subj"] = Subject
            d["film"] = ord(film) - 96
            d["ch"] = channel + 1
            d["labels"] = ratings.loc[Subject - 1][film + ".mp4"]
            d.to_excel(
                "features_spectral_new/S{}_F_{}_{}_.xlsx".format(
                    Subject, film, ch_names[channel]
                ),
                index=False,
            )
            data_all.append(d)
            first += 1
            second += 2
            ind += 1

  0%|          | 0/1 [00:00<?, ?it/s]

In [15]:
data_all = pd.concat(data_all)
data_all

,Theta,Alpha,Beta,Theta_ratio,Alpha_ratio,Beta_ratio,TotalAbsPow,Theta_watch-relax,Alpha_watch-relax,Beta_watch-relax,Beta/Alpha,Beta/(Alpha + Theta),Beta_watch/Alpha_watch,Beta_watch/(Alpha_watch + Theta_watch),Beta_watch/Alpha_watch - Beta_relax/Alpha_relax,Beta_watch/(Alpha_watch + Theta_relax) - Beta_relax/(Alpha_relax + Theta_relax),Subj,film,ch,labels
0,2.401640,2.783825,2.914699,0.318677,0.340933,0.340389,8.088658,-1.426278,0.805397,-0.424500,-0.527069,0.683705,1.047013,0.562090,-0.640792,-0.013004,4,a,Fz,8.0
1,1.848014,4.860342,3.861837,0.180087,0.491270,0.328643,10.238296,-0.034008,1.992814,-0.008136,-0.004083,-0.004153,0.794561,0.575676,-0.555024,-0.239133,4,a,F3,8.0
2,5.193237,4.188857,5.373905,0.372848,0.289807,0.337345,15.082173,-2.212639,-0.432822,-1.033615,2.388083,0.390713,1.282905,0.572783,-0.103500,0.040046,4,a,F7,8.0
3,3.109635,6.614255,5.331608,0.211328,0.455727,0.332945,14.337754,0.044150,2.854654,-0.113623,-0.039803,-0.039196,0.806078,0.548300,-0.642275,-0.249526,4,a,C3,8.0
4,3.711169,2.691938,3.848378,0.370692,0.267805,0.361503,10.278948,-1.303629,-0.368502,-0.990704,2.688463,0.592480,1.429594,0.601017,-0.151578,0.001768,4,a,T7,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,3.575201,1.851132,3.678716,0.387283,0.206470,0.406246,9.303020,0.170065,-0.803866,-1.172820,1.458974,1.850454,1.987280,0.677938,0.159958,-0.122628,4,h,Cz,6.0
140,4.967528,4.148056,6.345562,0.318008,0.279010,0.402982,15.554973,1.112179,-0.495866,-0.648755,1.308326,-1.052640,1.529768,0.696122,0.023645,-0.126809,4,h,C4,6.0
141,6.636565,5.209866,10.797049,0.290776,0.230819,0.478405,23.069312,1.583170,-0.458713,-4.950189,10.791478,-4.402292,2.072424,0.911418,-0.705563,-0.557270,4,h,T8,6.0
142,4.556124,3.119000,6.451290,0.322952,0.221992,0.455056,14.506383,0.565723,0.190684,-1.620096,-8.496252,-2.141831,2.068384,0.840545,-0.687939,-0.326056,4,h,F4,6.0


In [17]:
data_all.to_excel("Spectral.xlsx", index=False)

In [25]:
ch_names = [
    "Fz",
    "F3",
    "F7",
    "C3",
    "T7",
    "Pz",
    "P3",
    "P7",
    "O1",
    "Oz",
    "O2",
    "P4",
    "P8",
    "Cz",
    "C4",
    "T8",
    "F4",
    "F8",
]

features_all = pd.DataFrame()

n_subjects = 20

n_films = 8

n_channels = 18

for Subject in tqdm_notebook(range(1, n_subjects + 1)):

    if Subject == 5 or Subject == 21:
        continue

    for film in ["a", "b", "c", "d", "e", "f", "g", "h"]:

        for channel in range(n_channels):

            yasa = pd.read_excel(
                "features_spectral/S{}_F_{}_{}_.xlsx".format(
                    int(Subject), film, ch_names[channel]
                )
            )

            neurokit = pd.read_excel(
                "features_neurokit/S_{}_F_{}_{}_.xlsx".format(
                    int(Subject), film, ch_names[channel]
                ),
                engine="openpyxl",
            )

            df = pd.concat([yasa, neurokit], axis=1)

            features_all = pd.concat([features_all, df], axis=0)

  0%|          | 0/20 [00:00<?, ?it/s]

In [26]:
features_all

,Theta,Alpha,Beta,Theta_ratio,Alpha_ratio,Beta_ratio,TotalAbsPow,Theta_watch-relax,Alpha_watch-relax,Beta_watch-relax,...,PFD,RR,SFD,SVDEn,ShanEn,SpEn,Subj,film,ch,labels
0,5.478965,2.526810,2.764925,0.507505,0.234057,0.258438,10.995244,0.459282,0.486312,0.426391,...,1.003339,0.075773,1.608428,0.535017,15.271892,0.630001,1,1,1,10
0,2.884534,1.913928,2.768027,0.390887,0.252898,0.356215,7.767308,-0.600457,0.504085,0.141512,...,1.003882,0.108728,1.616296,0.592474,15.271639,0.690453,1,1,2,10
0,7.166805,4.005514,7.978071,0.382585,0.212711,0.404704,20.006198,0.683279,0.348539,2.843236,...,1.003305,0.085986,1.608730,0.554813,15.271892,0.578646,1,1,3,10
0,2.694183,2.027418,3.852692,0.318860,0.240955,0.440185,8.883501,0.197800,0.543780,1.163700,...,1.003822,0.113499,1.623063,0.599554,15.271892,0.585646,1,1,4,10
0,6.281124,4.022892,8.421597,0.332227,0.218676,0.449097,19.067099,1.717636,0.629256,3.446314,...,1.004642,0.168478,1.645645,0.665271,15.272094,0.680171,1,1,5,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,4.819919,2.735769,3.480329,0.445228,0.241847,0.312926,11.573306,0.146283,0.010476,-0.631298,...,1.002190,0.032390,1.538414,0.412120,15.571695,0.525844,20,8,14,6
0,3.433117,1.952647,7.195257,0.273939,0.158074,0.567987,12.843455,0.290486,-0.188057,2.239323,...,1.003640,0.088322,1.600959,0.559143,15.571695,0.522496,20,8,15,6
0,5.010104,2.711312,6.274102,0.362032,0.195539,0.442428,14.477907,1.689780,-0.323697,1.344576,...,1.003595,0.089069,1.615041,0.560490,15.571900,0.621396,20,8,16,6
0,1.430647,1.241856,8.130359,0.137981,0.108538,0.753481,10.736331,-0.087192,0.081011,3.753380,...,1.006014,0.267146,1.682560,0.743279,15.571695,0.614844,20,8,17,6
